In [1]:
from langchain_core.documents import Document
documents = [
    Document(
        page_content = "Dogs are great companions, known for their loyality and friendliness",
        metadata = {"source":"mammal-pets-doc"},
    ),
    Document(
        page_content = "Cats are independent pets that often enjoy their own space",
        metadata = {"source":"mammal-pets-doc"},
    ),
    Document(
        page_content = "Goldfish are popular pets for the begineers, required relatively simple care ",
        metadata = {"source":"fish-pets-doc"},
    ),
    Document(
        page_content = "Parrots are the intelligent birds couple of mimicking human speech",
        metadata = {"source":"bird-pets-doc"},
    ),
]
documents

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendliness'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for the begineers, required relatively simple care '),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are the intelligent birds couple of mimicking human speech')]

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"] = os.getenv("HF_TOKEN")
llm = ChatGroq(groq_api_key = groq_api_key, model = "Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10d431210>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10d431d80>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [3]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
embeddings = HuggingFaceEmbeddings(model_name = "all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents, embedding= embeddings)
vectorstore

/Users/vamshikrishna/Documents/GenAI/Langchain/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/vamshikrishna/Documents/GenAI/Langchain/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
vectorstore.similarity_search("Cats")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendliness'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are the intelligent birds couple of mimicking human speech'),
 Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for the begineers, required relatively simple care ')]

In [5]:
#Async query
await vectorstore.asimilarity_search("Cats")
vectorstore.similarity_search_with_score("Cats")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space'),
  0.8869501352310181),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendliness'),
  1.485456109046936),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are the intelligent birds couple of mimicking human speech'),
  1.502529263496399),
 (Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for the begineers, required relatively simple care '),
  1.687563180923462)]

In [6]:
#Retrivers
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
retriver = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriver.batch(["Cats","Dogs"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendliness')]]

In [7]:
#Vectorstore implement as_retriver
retriver = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":1}
)
retriver.batch(["Cats", "Dogs"])


[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space')],
 [Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyality and friendliness')]]

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message = """
Answer the question using the provided context only:
{question}
context:
{context}
"""
prompt = ChatPromptTemplate.from_messages([("human", message)])
rag_chain = {"context":retriver, "question":RunnablePassthrough()}|prompt|llm
response = rag_chain.invoke("tell me about cats")
response.content

'According to the provided context, cats are independent pets that often enjoy their own space.'